# The 2-IP model (implicit formulation)

$$
\begin{array}{crrcrrrl}
\max z_1 & = & x_1 & + & x_2  \\
\min z_2 & = & x_1 & + & 3x_2 \\
s.t. & \\
    &&  2x_1 & + &  3x_2 & \le & 30 \\
    &&  3x_1 & + &  2x_2 & \le & 30 \\
    &&   x_1 & - &   x_2 & \le & 5.5 \\
    &&   x_1 & , &   x_2 & \in & \mathbb{N}
\end{array}
$$

------

# The JuMP model

In [ ]:
using JuMP, HiGHS
import MultiObjectiveAlgorithms as MOA

In [ ]:
c1 = [1,1]; c2 = -1 * [1,3]; c = vcat(c1',c2')
a = [2 3; 3 2; 1 -1]
b = [30, 30, 5.5]
m,n = size(a)
p = size(c,1)

In [ ]:
md = Model( )

@variable(md, x[1:n]≥0, Int)
@expression(md, z[k=1:p], sum(c[k,j]*x[j] for j=1:n))
@objective(md, Max, [z[k] for k=1:p])
@constraint(md, ct[i=1:m], sum(a[i,j]*x[j] for j=1:n) ≤ b[i])

print(md)

In [ ]:
set_optimizer(md,()->MOA.Optimizer(HiGHS.Optimizer))
set_attribute(md, MOA.Algorithm(), MOA.EpsilonConstraint())
set_silent(md)
optimize!(md)

In [ ]:
for i in 1:result_count(md)
    z_opt = [objective_value(md; result = i)[k] for k=1:p]
    x_opt = value.(x; result = i)
end

## Plotting $Y_N$

In [ ]:
using Plots

In [ ]:
Plots.scatter(
    [value(z[1]; result = i) for i in 1:result_count(md)],
    [-1 * value(z[2]; result = i) for i in 1:result_count(md)];
    xlabel = "objective 1",
    ylabel = "objective 2",
    title = "objective space",
    legend = false,
    xlims = (0,25),
    ylims = (0,25),
    aspect_ratio=:equal,
)